In [157]:
# 패키지 로딩하기
import numpy as np
import tensorflow as tf
from PIL import Image
import os, glob

In [390]:
# 데이터 불러오는 함수 만들기 : 양창원님의 코드를 인용함
def load_data(img_path):
    # 가위 : 0, 바위 : 1, 보 : 2
    
    number_of_data = 0 
    for file in glob.iglob(img_path + '/rock/*.jpg'):
        number_of_data += 1
    for file in glob.iglob(img_path + '/scissor/*.jpg'):
        number_of_data += 1
    for file in glob.iglob(img_path + '/paper/*.jpg'):
        number_of_data += 1
    # 위 반복문을 실행하면 전체 이미지 파일 개수를 알 수 있다. 
    
    img_size = 28
    color    = 3
    
    #이미지 데이터와 라벨(가위 : 0, 바위 : 1, 보 : 2) 데이터를 담을 행렬(matrix) 영역을 생성합니다.
    imgs   = np.zeros(number_of_data * img_size * img_size * color, dtype = np.int32).reshape(number_of_data, img_size, img_size,color)
    labels = np.zeros(number_of_data, dtype = np.int32)

    idx = 0
    for file in glob.iglob(img_path + '/scissor/*.jpg'):
        img             = np.array(Image.open(file), dtype = np.int32)
        imgs[idx,:,:,:] = img    # 데이터 영역에 이미지 행렬을 복사
        labels[idx]     = 0      # 가위 : 0
        idx             = idx + 1

    for file in glob.iglob(img_path + '/rock/*.jpg'):
        img             = np.array(Image.open(file), dtype = np.int32)
        imgs[idx,:,:,:] = img    # 데이터 영역에 이미지 행렬을 복사
        labels[idx]     = 1      # 바위 : 1
        idx             = idx + 1       
    
    for file in glob.iglob(img_path + '/paper/*.jpg'):
        img             = np.array(Image.open(file), dtype = np.int32)
        imgs[idx,:,:,:] = img    # 데이터 영역에 이미지 행렬을 복사
        labels[idx]     = 2      # 보 : 2
        idx             = idx + 1
        
    print("이미지 개수는",idx,"입니다.")
    return imgs, labels

In [391]:
# 모든 데이터는 Slack에 양창원님께서 올려주신 15000개의 데이터를 사용했다.
# 트레이닝 데이터 : 14700개, 테스트 데이터 : 300개

In [392]:
# 훈련데이터 불러오기
image_dir_path     = os.getenv("HOME") + "/aiffel/dataset/*"
(x_train, y_train) = load_data(image_dir_path)

이미지 개수는 14700 입니다.


In [393]:
# test데이터 불러오기
image_dir_path     = os.getenv("HOME") + "/aiffel/data4"
(x_test, y_test)   = load_data(image_dir_path)

이미지 개수는 300 입니다.


In [394]:
# 데이터 정규화하기
x_train_normal       = x_train / 255.0
x_test_normal        = x_test  / 255.0

In [395]:
# 정규화 확인하기 : 0 ~ 1 사이의 값을 가져야 함
print(np.min(x_train_normal), np.max(x_train_normal))   # 훈련 데이터
print(np.min(x_test_normal), np.max(x_test_normal))     # test 데이터

0.0 1.0
0.0 0.8196078431372549


In [396]:
# 불러온 데이터 shape 확인하기
# (데이터 갯수, 가로, 세로, 깊이(색깔이 흑백이면 1, 컬러면 RGB를 뜻하는 3)

# 훈련 데이터
print(x_train_normal.shape)
print(y_train.shape)

# test 데이터
print(x_test_normal.shape)
print(y_test.shape)

(14700, 28, 28, 3)
(14700,)
(300, 28, 28, 3)
(300,)


In [397]:
# 모델 만들기

# 모델 생성
tf_model = tf.keras.Sequential()
# 계층 1
tf_model.add(tf.keras.layers.Conv2D(filters = 64, kernel_size = (3, 3), input_shape = (28, 28, 3), activation = "relu"))
tf_model.add(tf.keras.layers.MaxPooling2D(pool_size = (1, 1)))
tf_model.add(tf.keras.layers.Dropout(0.3))
# 계층 2
tf_model.add(tf.keras.layers.Conv2D(filters = 64, kernel_size = (3, 3), activation = "relu"))
tf_model.add(tf.keras.layers.MaxPooling2D(pool_size = (2, 2)))
tf_model.add(tf.keras.layers.Dropout(0.3))
# 계층 3
tf_model.add(tf.keras.layers.Conv2D(filters = 128, kernel_size = (3, 3), activation = "relu"))
tf_model.add(tf.keras.layers.MaxPooling2D(pool_size = (2, 2)))
tf_model.add(tf.keras.layers.Dropout(0.3))
# 계층 4
tf_model.add(tf.keras.layers.Conv2D(filters = 128, kernel_size = (3, 3), activation = "relu"))
tf_model.add(tf.keras.layers.MaxPooling2D(pool_size = (2, 2)))
tf_model.add(tf.keras.layers.Dropout(0.3))
# 계층 5
tf_model.add(tf.keras.layers.Flatten())
tf_model.add(tf.keras.layers.Dropout(0.3))
tf_model.add(tf.keras.layers.Dense(units = 512, activation = "relu"))
tf_model.add(tf.keras.layers.Dense(units = 3, activation = "softmax"))

# 모델 확인하기
tf_model.summary()

Model: "sequential_39"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_156 (Conv2D)          (None, 26, 26, 64)        1792      
_________________________________________________________________
max_pooling2d_156 (MaxPoolin (None, 26, 26, 64)        0         
_________________________________________________________________
dropout_176 (Dropout)        (None, 26, 26, 64)        0         
_________________________________________________________________
conv2d_157 (Conv2D)          (None, 24, 24, 64)        36928     
_________________________________________________________________
max_pooling2d_157 (MaxPoolin (None, 12, 12, 64)        0         
_________________________________________________________________
dropout_177 (Dropout)        (None, 12, 12, 64)        0         
_________________________________________________________________
conv2d_158 (Conv2D)          (None, 10, 10, 128)     

In [398]:
# 모델 Loss, Optimizer, Metrics 설정하기
tf_model.compile(loss = "sparse_categorical_crossentropy", optimizer = tf.keras.optimizers.Adam(lr = 0.001), metrics = ["accuracy"])

In [399]:
# 모델 훈련하기
hostory = tf_model.fit(x_train_normal, y_train, epochs = 15)

Epoch 1/15
460/460 [==============================] - 2s 3ms/step - loss: 1.0982 - accuracy: 0.3422
Epoch 2/15
460/460 [==============================] - 2s 3ms/step - loss: 0.9726 - accuracy: 0.4978
Epoch 3/15
460/460 [==============================] - 1s 3ms/step - loss: 0.7222 - accuracy: 0.6812
Epoch 4/15
460/460 [==============================] - 2s 3ms/step - loss: 0.5564 - accuracy: 0.7638
Epoch 5/15
460/460 [==============================] - 2s 3ms/step - loss: 0.4477 - accuracy: 0.8142
Epoch 6/15
460/460 [==============================] - 2s 3ms/step - loss: 0.3917 - accuracy: 0.8439
Epoch 7/15
460/460 [==============================] - 2s 3ms/step - loss: 0.3472 - accuracy: 0.8639
Epoch 8/15
460/460 [==============================] - 2s 3ms/step - loss: 0.3180 - accuracy: 0.8731
Epoch 9/15
460/460 [==============================] - 2s 3ms/step - loss: 0.2827 - accuracy: 0.8867
Epoch 10/15
460/460 [==============================] - 2s 3ms/step - loss: 0.2767 - accuracy: 0.8936

In [403]:
# 모델 시험하기 : accuracy가 0.6 이상이 나와야 함.
evaluation1 = tf_model.evaluate(x_test_normal, y_test)
# 점수가 제일 좋게 나온 data폴더를 test 데이터로 사용하였음.
# 총 19개 폴더(각 폴더당 사진은 300개)의 결과는 아래에 실어놓음.

10/10 [==============================] - 0s 2ms/step - loss: 0.0089 - accuracy: 1.0000


## 회고

### 1. 어려웠던 점 및 모호한 점

#### - 모델은 잘 학습되나 test 데이터가 바뀔 때 마다 결과가 상이함(최고 100%, 최소 31.6%)
#### - test 데이터가 바뀔때 마다 결과가 다른 이유는 눈으로 봐서는 잘 모르겠음
#### - 19개 중 15개가 60%를 넘긴것으로 보아 추측컨데 나머지 4개의 데이터 셋은 트레이닝 셋과 많이 다른 것으로 생각됨
#### - 그 차이는 눈으로 보고 추측하기로는 손과 나머지 배경의 경계가 뚜렷하게 나온 사진들을 더 잘 분류하는것으로 생각함

### 2. 루브릭 평가를 위해 시도한 것

#### - 좋은 건지 모르겠으나 최대한 계층을 많이 넣었음.
#### - 학습을 100번 시켜보고 어느 횟수가 적당한지 본 다음 다시 모델을 만들어서 학습 시킴(그 결과 실전에서는 15번 학습시킴)
#### - 오버피팅 되는것을 방지하기 위해 pooling과 dropout을 해줌. kernel_initializer란 것도 있던거 같으나 이해하지 못해서 안 씀.

### 3. 아쉬운 점

#### - 아직 코드를 보고 어떤 것이 어떤 원리로 작동하는지 알지 못해서 내 코딩이 어디가 부족한지 찾지 못함
#### - 구글링을 해도 무슨말을 하는것인지 모르겠으며, 어떤 것은 나보다 계층이 적은데도 잘 되었다는 블로그가 있어서 더 아쉬움
#### - 아직 오버피팅을 피하는 방법을 제대로 몰라서 학습횟수와 계층 수와 드랍아웃으로만 조절한 것 같아서 오버피팅 여부를 확신할 수 없음

### 4. 궁금증을 해결하기 위해 19개의 폴더들을 각각 전부 test 데이터로 설정해서 돌려봄

#### - 당연하겠지만 test 데이터로 뽑힌 폴더는 상위폴더에서 빼내어 다른곳에 넣고 트레이닝 데이터에 포함되지 않게 함
#### - 때문에 데이터 불러오기, 정규화, 모델 생성, 학습, 결과보기를 총 20번 함
#### - 결과는 흥미롭지만 왜 차이가 나는지는 모르겠음
#### - 19개중 15개가 0.6을 넘은것으로 보아 데이터문제라고 말하고 싶은것이 사실임

In [ ]:
# 모델 시험하기 : accuracy가 0.6 이상이 나와야 함.

In [217]:
# data1을 test데이터로 한 경우
evaluation1 = tf_model.evaluate(x_test_normal, y_test)

10/10 [==============================] - 0s 2ms/step - loss: 1.4793 - accuracy: 0.5500


In [207]:
# data2을 test데이터로 한 경우
evaluation1 = tf_model.evaluate(x_test_normal, y_test) 

10/10 [==============================] - 0s 2ms/step - loss: 1.2303 - accuracy: 0.6267


In [227]:
# data3을 test데이터로 한 경우
evaluation1 = tf_model.evaluate(x_test_normal, y_test) 

10/10 [==============================] - 0s 2ms/step - loss: 0.1226 - accuracy: 0.9800


In [237]:
# data4을 test데이터로 한 경우
evaluation1 = tf_model.evaluate(x_test_normal, y_test) 

10/10 [==============================] - 0s 2ms/step - loss: 0.0136 - accuracy: 1.0000


In [247]:
# data5을 test데이터로 한 경우
evaluation1 = tf_model.evaluate(x_test_normal, y_test) 

10/10 [==============================] - 0s 2ms/step - loss: 1.3790 - accuracy: 0.5867


In [257]:
# data6을 test데이터로 한 경우
evaluation1 = tf_model.evaluate(x_test_normal, y_test) 

10/10 [==============================] - 0s 2ms/step - loss: 0.4220 - accuracy: 0.8467


In [267]:
# data7을 test데이터로 한 경우
evaluation1 = tf_model.evaluate(x_test_normal, y_test) 

10/10 [==============================] - 0s 2ms/step - loss: 0.0537 - accuracy: 0.9933


In [277]:
# data8을 test데이터로 한 경우
evaluation1 = tf_model.evaluate(x_test_normal, y_test) 

10/10 [==============================] - 0s 2ms/step - loss: 1.2453 - accuracy: 0.3167


In [287]:
# data9을 test데이터로 한 경우
evaluation1 = tf_model.evaluate(x_test_normal, y_test) 

10/10 [==============================] - 0s 2ms/step - loss: 0.1197 - accuracy: 0.9733


In [297]:
# data10을 test데이터로 한 경우
evaluation1 = tf_model.evaluate(x_test_normal, y_test) 

10/10 [==============================] - 0s 2ms/step - loss: 0.2668 - accuracy: 0.9400


In [307]:
# data11을 test데이터로 한 경우
evaluation1 = tf_model.evaluate(x_test_normal, y_test) 

10/10 [==============================] - 0s 2ms/step - loss: 0.4124 - accuracy: 0.8767


In [317]:
# data12을 test데이터로 한 경우
evaluation1 = tf_model.evaluate(x_test_normal, y_test) 

10/10 [==============================] - 0s 2ms/step - loss: 1.0664 - accuracy: 0.6767


In [327]:
# data13을 test데이터로 한 경우
evaluation1 = tf_model.evaluate(x_test_normal, y_test) 

10/10 [==============================] - 0s 2ms/step - loss: 1.5280 - accuracy: 0.6967


In [337]:
# data14을 test데이터로 한 경우
evaluation1 = tf_model.evaluate(x_test_normal, y_test) 

10/10 [==============================] - 0s 2ms/step - loss: 0.7829 - accuracy: 0.6633


In [347]:
# data15을 test데이터로 한 경우
evaluation1 = tf_model.evaluate(x_test_normal, y_test) 

10/10 [==============================] - 0s 2ms/step - loss: 1.1793 - accuracy: 0.6467


In [357]:
# data16을 test데이터로 한 경우
evaluation1 = tf_model.evaluate(x_test_normal, y_test) 

10/10 [==============================] - 0s 2ms/step - loss: 0.9208 - accuracy: 0.5667


In [367]:
# data17을 test데이터로 한 경우
evaluation1 = tf_model.evaluate(x_test_normal, y_test) 

10/10 [==============================] - 0s 2ms/step - loss: 0.0622 - accuracy: 0.9767


In [378]:
# data18을 test데이터로 한 경우
evaluation1 = tf_model.evaluate(x_test_normal, y_test) 

10/10 [==============================] - 0s 2ms/step - loss: 0.1947 - accuracy: 0.8700


In [389]:
# data19을 test데이터로 한 경우
evaluation1 = tf_model.evaluate(x_test_normal, y_test) 

10/10 [==============================] - 0s 2ms/step - loss: 0.5040 - accuracy: 0.7933
